In [102]:
import pandas as pd
import requests
import time, os
import re
import json
import pymysql

In [103]:
db = pymysql.connect(host='54.166.249.1', port=3306, user='damdam', passwd='damdam2020', db='damdam', charset="utf8mb4", autocommit=True, cursorclass=pymysql.cursors.DictCursor)
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

In [100]:
import csv
f = open('삼성전자 노트북 플러스 NT550XCR-AD1A (SSD 128GB).csv','r',encoding='utf-8-sig')
r = csv.reader(f)
empty = []
for i in r:
    empty.append(i)

print(empty[0])
f.close()
#['제품명','카테고리','브랜드','가격','아이디','실제 사이트','리뷰 등록 날짜','본문','리뷰 점수','리뷰가 올라와있는 url','이미지url']

['노트북 플러스 NT550XCR-AD1A (SSD 128GB)', '노트북', '삼성전자', '433000', 'kk****', 'G마켓', '2020.06.06', '처음에 누가 사용하던 노트북을 받아서 두번째로 새 노트북 받았네요. 노트북은 저렴하게 잘 산 것 같지만 기존에 사용하던 노트북이 고장나서 급하게 주문한건데 잘못 받으니 굉장히 화가 나더라구요. 리퍼제품을 제가 새 제품 파는걸로 착각하고 구매했나 싶었습니다. 암튼 새로 받기는 했지만 제대로 상품 확인도 안하고 보내는 건 시정될 필요가 있다고 봅니다. 구매하신분들도 받으시면 바로 확인해보세요.', '60점', 'http://prod.danawa.com/info/?pcode=11093343&cate=11234543', 'http://img.danawa.com/prod_img/500000/343/093/img/11093343_1.jpg?shrink=500:500&_v=20200514102027', '0']


In [101]:
f = open('삼성전자 노트북 플러스 NT550XCR-AD1A (SSD 128GB).csv','w',encoding='utf-8-sig')
w = csv.writer(f)
for i in empty:
    if i[8] =='20점':
        i[8] = 1
    elif i[8] == '40점':
        i[8] = 2
    elif i[8] == '60점':
        i[8] = 3
    elif i[8] == '80점':
        i[8] = 4
    else:
        i[8] = 5
    w.writerow(i)

f.close()

In [43]:
#카테고리 함수
def insertCategory(c_name):
    with db.cursor() as cursor:
        insertCategorySql = f"INSERT INTO category (c_name) values('{c_name}');"
        cursor.execute(insertCategorySql)

def getCategory(c_name):
    with db.cursor() as cursor:
        getCategorySql = f"SELECT c_id FROM category WHERE c_name = '{c_name}';"
        cursor.execute(getCategorySql)
        result = cursor.fetchone()
        return result["c_id"]


In [106]:
getCategory('노트북')

2

In [89]:
#아이템 함수
def insertItem(i_name,detail,c_id,brand_name,img_url,detail_url,site,price):
    with db.cursor() as cursor:
        insertItemSql = f"INSERT INTO item (i_name, detail, c_id, brand_name, img_url, detail_url, site, price) values('{i_name}','{detail}','{c_id}','{brand_name}','{img_url}','{detail_url}','{site}','{price}');"
        try:
            cursor.execute(insertItemSql)
        except:
            i_name = i_name.encode('ascii','ignore').decode('ascii')
            brand_name = brand_name.encode('ascii','ignore').decode('ascii')
            insertItemSql = f"INSERT INTO item (i_name, detail, c_id, brand_name, img_url, detail_url, site, price) values('{i_name}','{detail}','{c_id}','{brand_name}','{img_url}','{detail_url}','{site}','{price}');"
            cursor.execute(insertItemSql)

def getItem(i_name):
    with db.cursor() as cursor:
        getItemSql = f"SELECT i_id FROM item WHERE i_name = '{i_name}';"
        cursor.execute(getItemSql)
        result = cursor.fetchone()
        if result == None:
            return None
        else:
            return result["i_id"]

In [105]:
empty[0]

['노트북 플러스 NT550XCR-AD1A (SSD 128GB)',
 '노트북',
 '삼성전자',
 '433000',
 'kk****',
 'G마켓',
 '2020.06.06',
 '처음에 누가 사용하던 노트북을 받아서 두번째로 새 노트북 받았네요. 노트북은 저렴하게 잘 산 것 같지만 기존에 사용하던 노트북이 고장나서 급하게 주문한건데 잘못 받으니 굉장히 화가 나더라구요. 리퍼제품을 제가 새 제품 파는걸로 착각하고 구매했나 싶었습니다. 암튼 새로 받기는 했지만 제대로 상품 확인도 안하고 보내는 건 시정될 필요가 있다고 봅니다. 구매하신분들도 받으시면 바로 확인해보세요.',
 3,
 'http://prod.danawa.com/info/?pcode=11093343&cate=11234543',
 'http://img.danawa.com/prod_img/500000/343/093/img/11093343_1.jpg?shrink=500:500&_v=20200514102027',
 '0']

In [107]:
insertItem(empty[0][0],'',getCategory(empty[0][1]),empty[0][2],empty[0][10],empty[0][9],empty[0][5],empty[0][3])

In [94]:
#리뷰함수
def insertReview(star,content,writer,regdate,i_id,site):
    with db.cursor() as cursor:
        content = re.sub('[\"\']','',content)
        insertReviewSql = f"INSERT INTO review (star, content, writer, regdate, i_id, site) values(\"{star}\",\"{content}\",\"{writer}\",\"{regdate}\",\"{i_id}\",\"{site}\");"
        try:
            cursor.execute(insertReviewSql)
        except:
            content = content.encode('ascii','ignore').decode('ascii')
            writer = writer.encode('ascii','ignore').decode('ascii')
            insertReviewSql = f"INSERT INTO review (star, content, writer, regdate, i_id, site) values(\"{star}\",\"{content}\",\"{writer}\",\"{regdate}\",\"{i_id}\",\"{site}\");"
            cursor.execute(insertReviewSql)

def getReview(r_id):
    with db.cursor() as cursor:
        getReviewSql = f"SELECT r_id FROM review WHERE site_r_id = '{r_id}';"
        cursor.execute(getReviewSql)
        result = cursor.fetchone()
        return result["r_id"]

In [108]:
for i in range(0,len(empty)):
    insertReview(empty[i][8],empty[i][7],empty[i][4],empty[i][6],getItem(empty[i][0]),empty[i][5])